In [ ]:
# Imports
import numpy as np
import pandas as pd
import math
import importlib as imp
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.losses import Huber
from keras.utils import to_categorical

In [ ]:
# User module imports
from utils import district_daily_data as dd
from utils import get_lockdown as gl
dd = imp.reload(dd)
gl = imp.reload(gl)

In [ ]:
# Flags
lstm_model = True

In [ ]:
# Directory variables
data_dir = 'data/'

In [ ]:
# Read state data
df_state = pd.read_csv(data_dir + 'state-date-total-data.csv')
arr_state = df_state.to_numpy() # still reversed
arr_state = np.flipud(arr_state) # now taken data from day-1 to day-52; but still daily cases
arr_state = np.cumsum(arr_state, axis=0) # now cumulative cases till day 52
np.savetxt(data_dir + 'state-date-total-data-cumulative.csv', arr_state.astype(int), fmt='%i', delimiter=",")

In [ ]:
# Read population density data
df_population_density = pd.read_csv(data_dir + 'district_wise_population_density.csv')
np_population_density = df_population_density.to_numpy() 
data_found_count = 0 # no of districts for which we have population density data

def get_district_population_density(d):
    global data_found_count
    dist_pop_density = -2
    for i_cn in range(len(np_population_density)):
        if(np_population_density[i_cn][1].lower().count(d.lower().strip()) > 0):
            dist_pop_density = max(float(np_population_density[i_cn][7]), dist_pop_density)
    if(dist_pop_density <= 0): # print(d) # district not matched || area not found || population data missing
        dist_pop_density = 368 # population density of INDIA
    else:
        data_found_count = data_found_count + 1
    return dist_pop_density

In [ ]:
# Read district data
districts = dd.get_all_districts()
dist_series = []  # [(start_date, series), (start_date, series), ...]
max_number = 0

# Note: start_date might itself be a feature
for d in districts:
    if d == "Mumbai" or d == "Thane" or d == "Delhi" or d == "Lucknow":
#     if d == "Mumbai":
        d_start_date = dd.get_infection_start(d)
        district_pop_density = get_district_population_density(d)
        district_time_series = dd.get_district_time_series(d, d_start_date)
        lockdown_time_series = gl.get_lockdown_series(d, d_start_date, len(district_time_series))
        dist_series.append((d, district_time_series, lockdown_time_series, district_pop_density))
        
        # Update district max to scale all numbers
        district_max = max(district_time_series)
        if district_max > max_number:
            max_number = district_max

print(dist_series)
print("data_found_count:", data_found_count,  " tot dists:", len(districts))

In [ ]:
feature_range = 5

# Transform using MinMaxScaler
def fit_transform(series):
    global max_number
    series = np.array(list(map(lambda x: x/max_number*feature_range, series)))
    return series

# Revert the transform to get actual series
def inverse_transform(series):
    global max_number
    series = np.array(list(map(lambda x: np.rint(x*max_number/feature_range), series)))
    return series

In [ ]:
# Get separate train and test sets with data points from each of the districts
def divide_series(dist_series, train_percent, look_ahead=1):
    # Construct train and test data and fit Support Vector Regression
    x_train = []
    x_test = []
    y_train = []
    y_test = []
    # Construct a list of district wise input features and outputs
    dist_set = []
    episode_length = 14
    count = 0
    for tup in dist_series:
        # Series for infected numbers
        series = tup[1]
        a = np.array(series)
        series = np.reshape(a, (a.shape[0], 1))
        series = fit_transform(series)
        
        # Series for lockdown levels and population density
        ld_series = to_categorical(tup[2])
        dist_pop_density = tup[3]
        pd_series = np.repeat(tup[3], len(series))
        pd_series = np.reshape(pd_series, (pd_series.shape[0], 1))
        
        num_episodes = len(series) - episode_length + 1
        if num_episodes < 2: continue

        dist_x = []
        dist_y = []
        for _in in range(num_episodes-look_ahead):
            # Concatenate infected numbers and lockdown levels
            multivar_x = np.concatenate((series[_in:_in+episode_length], ld_series[
                _in:_in+episode_length]), axis=1)
            # Also concatenate population density along with these numbers
            # multivar_x = np.concatenate((multivar_x, pd_series[_in:_in+episode_length]), axis=1)
            dist_x.append(multivar_x)
            dist_y.append(series[_in+episode_length+look_ahead-1])
        
        # Add to Dsitrict wise set
        dist_set.append((tup[0], dist_x, dist_y))
        
        # Training and testing set
        train_length = int(train_percent*len(dist_x))
        x_train.extend(dist_x[:train_length])
        y_train.extend(dist_y[:train_length])
        x_test.extend(dist_x[train_length:-1])
        y_test.extend(dist_y[train_length:-1])

    x_train = np.array(x_train)
    x_test = np.array(x_test)
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    print (x_train.shape, x_test.shape)
    return (x_train, y_train), (x_test, y_test), dist_set

In [ ]:
look_back = 14
num_features = 7
train_percent = 0.75

if lstm_model:
    train, test, dist_set = divide_series(dist_series, train_percent=train_percent, look_ahead=6)

    # reshape input to be [samples, time steps, features]
    np.random.seed(7)
    trainX = np.reshape(train[0], (train[0].shape[0], train[0].shape[1], num_features))
    testX = np.reshape(test[0], (test[0].shape[0], test[0].shape[1], num_features))

    # create and fit the LSTM network
    model = Sequential()
    # model.add(LSTM(2, input_shape=(look_back, 1), return_sequences=True))
    model.add(LSTM(7, input_shape=(look_back, num_features)))
    model.add(Dense(1))
    model.compile(loss=Huber(delta=50), optimizer='adam')
    model.fit(trainX, train[1], epochs=150, batch_size=1, verbose=2)

    # make predictions
    trainPredict = model.predict(trainX)
    testPredict = model.predict(testX)

    # invert predictions
    trainPredict = inverse_transform(trainPredict)
    trainY = inverse_transform(train[1])
    testPredict = inverse_transform(testPredict)
    testY = inverse_transform(test[1])
    
    # calculate root mean squared error
    trainScore = math.sqrt(mean_squared_error(trainY[:,0], trainPredict[:,0]))
    print('Train Score: %.2f RMSE' % (trainScore))
    testScore = math.sqrt(mean_squared_error(testY[:,0], testPredict[:,0]))
    print('Test Score: %.2f RMSE' % (testScore))

    dataset = dist_series[0][1]
    a = np.array(dataset)
    dataset = a.reshape(a.shape[0], 1)

In [ ]:
# Plot graphs for each of the districts in the dist_series
for tup in dist_set:
    dist_x = np.array(tup[1])
    dist_y = np.array(tup[2])
    train_length = int(train_percent*len(dist_x))
    
    trainX = dist_x[:train_length]
    testX = dist_x[train_length:-1]
    trainY = dist_y[:train_length]
    testY = dist_y[train_length:-1]
    
    trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], num_features))
    testX = np.reshape(testX, (testX.shape[0], testX.shape[1], num_features))

    # make predictions
    trainPredict = model.predict(trainX)
    testPredict = model.predict(testX)

    # invert predictions
    trainPredict = inverse_transform(trainPredict)
    trainY = inverse_transform(trainY)
    testPredict = inverse_transform(testPredict)
    testY = inverse_transform(testY)

    print(trainY.shape, trainPredict.shape)
    print(testY.shape, testPredict.shape)

    x1 = np.arange(1, trainY.shape[0]+1)
    x2 = np.arange(trainY.shape[0]+1, trainY.shape[0]+1+testY.shape[0])

    print (tup[0] + ':')
    # Time Series
    plt.plot(x1, trainY)
    plt.plot(x1, trainPredict)
    plt.plot(x2, testY)
    plt.plot(x2, testPredict)
    plt.show()

    # Cumulative
    x = np.append(x1, x2)
    y_true = np.append(trainY, testY)
    y_pred = np.append(trainPredict, testPredict)
    plt.plot(x, np.cumsum(y_true))
    plt.plot(x, np.cumsum(y_pred))
    plt.show()